# Cleaning SF36 Data & scoring

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import seaborn as sns
import scipy
import re
import glob
import shutil
from datetime import date

In [2]:
# Import data
path = '/Users/mgg/Library/CloudStorage/GoogleDrive-matthieu.gallou.guyot@gmail.com/.shortcut-targets-by-id/1vVhXgDsltU_Yk0bJYMY4l0DVX_1VGkOd/Dakar/Q-SF36'

data_dict = {}

for file_name in os.listdir(path):
    if file_name.endswith('.csv') and re.match(r'SF36-\d{3}v\d{1}\.csv', file_name):
        file_path = os.path.join(path, file_name)
        df = pd.read_csv(file_path).T
        df.columns = df.iloc[0]  # Use the first row as column headers
        df = df[1:]  # Remove the first row after using it as headers
        df = df.reset_index(drop=True)  # Reset the index
        df = df.rename_axis(None, axis=1)
        df.insert(0, "ID", file_name)  # Insert 'ID' column with the file name
        data_dict[file_name] = df

data_dict = dict(sorted(data_dict.items()))

In [59]:
#data_dict['SF36-104v1.csv']

# Clean data = missing or out-of-range values, items reorganisation

In [3]:
# Check for missing values
for key, df in data_dict.items():
    missing_values = df[df.isna().any(axis=1)]
    if not missing_values.empty:
        for index, row in missing_values.iterrows():
            missing_columns = row.index[row.isna()]
            print(f"ID {key}: missing in {', '.join(missing_columns)}")

ID SF36-002v1.csv: missing in Q10a
ID SF36-028v1.csv: missing in Q4a, Q9a
ID SF36-034v1.csv: missing in Q10e
ID SF36-038v3.csv: missing in Q9e, Q10b
ID SF36-044v1.csv: missing in Q3a, Q3b, Q3c, Q4a, Q4b
ID SF36-046v1.csv: missing in Q3d, Q9b
ID SF36-055v3.csv: missing in Q9d
ID SF36-056v1.csv: missing in Q10h
ID SF36-059v1.csv: missing in Q9e
ID SF36-079v1.csv: missing in Q9e, Q9i, Q10h
ID SF36-083v1.csv: missing in Q9d
ID SF36-086v1.csv: missing in Q9d
ID SF36-087v1.csv: missing in Q10g
ID SF36-096v1.csv: missing in Q9h
ID SF36-101v1.csv: missing in Q3c
ID SF36-103v1.csv: missing in Q9i, Q11d
ID SF36-104v1.csv: missing in Q9e


In [4]:
total_missing_values = pd.concat([df for df in data_dict.values()]).isna().sum().sum()
total_items = sum(len(df.columns) for df in data_dict.values()) * len(data_dict)

num_subjects = sum(len(df) for df in data_dict.values())
percentage_missing_values = (total_missing_values / total_items) * 100
average_missing_values_per_subject = total_missing_values / num_subjects

print("Global Missing Value Statistics:")
print(f"  Total missing values: {total_missing_values}")
print(f"  Percentage of missing values compared to total items: {percentage_missing_values:.2f}%")
print(f"  Average missing values per subject: {average_missing_values_per_subject:.2f}")


Global Missing Value Statistics:
  Total missing values: 27
  Percentage of missing values compared to total items: 0.00%
  Average missing values per subject: 0.19


In [62]:
# All true. Old (modified):
# ID SF36-022v3.csv: missing in Q3b, Q3c, Q3d, Q4b, Q4c = seem to be 1 everywhere 
# ID SF36-031v1.csv: missing in Q3c, Q3d, Q4b, Q9b, Q9c, Q9g = where filled 

In [5]:
# Check for acherrante values (out of range)
value_bounds = {
    'Q1': (1, 5),
    'Q2': (1, 5),
    'Q3a': (1, 2),
    'Q3b': (1, 2),
    'Q3c': (1, 2),
    'Q3d': (1, 2),
    'Q4a': (1, 2),
    'Q4b': (1, 2),
    'Q4c': (1, 2),
    'Q5': (1, 5),
    'Q6': (1, 6),
    'Q7': (1, 5),
    'Q8': (1, 5),
    'Q9a': (1,3), 
    'Q9b': (1,3), 
    'Q9c': (1,3), 
    'Q9d': (1,3), 
    'Q9e': (1,3), 
    'Q9f': (1,3), 
    'Q9g': (1,3),
    'Q9h': (1,3), 
    'Q9i': (1,3), 
    'Q9j': (1,3), 
    'Q10a': (1,6), 
    'Q10b': (1,6), 
    'Q10c': (1,6), 
    'Q10d': (1,6), 
    'Q10e': (1,6), 
    'Q10f': (1,6),
    'Q10g': (1,6), 
    'Q10h': (1,6), 
    'Q10i': (1,6), 
    'Q11a': (1,5), 
    'Q11b': (1,5), 
    'Q11c': (1,5), 
    'Q11d': (1,5)    
}

for key, df in data_dict.items():
    print(f"Checking {key}:")
    for col, bounds in value_bounds.items():
        lower_bound, upper_bound = bounds
        out_of_bounds = df[(df[col] < lower_bound) | (df[col] > upper_bound)]
        if not out_of_bounds.empty:
            print(f"    Key: {key}, Column: {col}, Out of bounds values: {out_of_bounds}")

Checking SF36-001v1.csv:
Checking SF36-001v3.csv:
Checking SF36-002v1.csv:
Checking SF36-003v1.csv:
Checking SF36-003v3.csv:
Checking SF36-004v1.csv:
Checking SF36-004v3.csv:
Checking SF36-005v1.csv:
Checking SF36-005v3.csv:
Checking SF36-006v1.csv:
Checking SF36-007v1.csv:
Checking SF36-008v1.csv:
Checking SF36-009v1.csv:
Checking SF36-010v1.csv:
Checking SF36-011v1.csv:
Checking SF36-012v1.csv:
Checking SF36-013v1.csv:
Checking SF36-014v1.csv:
Checking SF36-020v1.csv:
Checking SF36-020v3.csv:
Checking SF36-022v1.csv:
Checking SF36-022v3.csv:
Checking SF36-023v1.csv:
Checking SF36-024v1.csv:
Checking SF36-024v3.csv:
Checking SF36-025v1.csv:
Checking SF36-025v3.csv:
Checking SF36-026v1.csv:
Checking SF36-026v3.csv:
Checking SF36-027v1.csv:
Checking SF36-027v3.csv:
Checking SF36-028v1.csv:
Checking SF36-029v1.csv:
Checking SF36-029v3.csv:
Checking SF36-030v1.csv:
Checking SF36-030v3.csv:
Checking SF36-031v1.csv:
Checking SF36-031v2.csv:
Checking SF36-031v3.csv:
Checking SF36-032v1.csv:


In [6]:
# Reorganise items
columns_to_prefix = ['Q1', 'Q2', 'Q3a', 'Q3b', 'Q3c', 'Q3d', 'Q4a', 'Q4b', 'Q4c', 'Q5', 'Q6', 'Q7', 'Q8', 
                     'Q9a', 'Q9b', 'Q9c', 'Q9d', 'Q9e', 'Q9f', 'Q9g', 'Q9h', 'Q9i', 'Q9j', 
                     'Q10a', 'Q10b', 'Q10c', 'Q10d', 'Q10e', 'Q10f', 'Q10g', 'Q10h', 'Q10i', 
                     'Q11a', 'Q11b', 'Q11c', 'Q11d']

mappings = {
    'Old_Q9': 'New_Q3',
    'Old_Q3': 'New_Q4',
    'Old_Q4': 'New_Q5',
    'Old_Q5': 'New_Q6',
    'Old_Q6': 'New_Q7',
    'Old_Q7': 'New_Q8',
    'Old_Q10': 'New_Q9',
    'Old_Q8': 'New_Q10'  
}

new_order = ['ID', '1', '2', '3a', '3b', '3c', '3d', '3e', '3f', '3g', '3h', '3i', '3j', 
             '4a', '4b', '4c', '4d', '5a', '5b', '5c', '6', '7', '8', 
             '9a', '9b', '9c', '9d', '9e', '9f', '9g', '9h', '9i', '10','11a', '11b', '11c', '11d']

for key, df in data_dict.items():
    for col in columns_to_prefix:
        df.rename(columns={col: 'Old_' + col}, inplace=True)
    
    for col in df.columns:
        for old, new in mappings.items():
            if col.startswith(old):
                new_col = col.replace(old, new)
                df.rename(columns={col: new_col}, inplace=True)
    
    df.columns = df.columns.str.replace(r'(Old|New)_Q', '', regex=True)
    df = df[new_order]

    data_dict[key] = df

In [65]:
#data_dict['SF36-104v1.csv']

# Scoring = recalibrate scores, calculating scales

In [7]:
# Reverse / recalibrate score for some items
replacement_dicts = {
    '1': {1: 5.0, 2: 4.4, 3: 3.4, 4: 2.0, 5: 1.0},
    '6': {1: 5, 2: 4, 3: 3, 4: 2, 5: 1},
    '7': {1: 6.0, 2: 5.4, 3: 4.2, 4: 3.1, 5: 2.2, 6: 1.0},
    '9a': {1: 6, 2: 5, 3: 4, 4: 3, 5: 2, 6: 1},
    '9e': {1: 6, 2: 5, 3: 4, 4: 3, 5: 2, 6: 1},
    '9d': {1: 6, 2: 5, 3: 4, 4: 3, 5: 2, 6: 1},
    '9h': {1: 6, 2: 5, 3: 4, 4: 3, 5: 2, 6: 1},
    '11b': {1: 5, 2: 4, 3: 3, 4: 2, 5: 1},
    '11d': {1: 5, 2: 4, 3: 3, 4: 2, 5: 1}
}

# Parcourir chaque DataFrame dans data_dict
for key, df in data_dict.items():
    for col, replacement_dict in replacement_dicts.items():
        if col in df.columns:
            df[col] = df[col].replace(replacement_dict)
            data_dict[key] = df
            
    if '7' in df.columns and '8' in df.columns:
        condition = df['7'].isna()
        replacement_dict_conditional = {
            1: np.where(condition, 6.0, 5),
            2: np.where(condition, 4.75, 4),
            3: np.where(condition, 3.5, 3),
            4: np.where(condition, 2.25, 2),
            5: np.where(condition, 1.0, 1)
        }
        df['8'] = df['8'].replace(replacement_dict_conditional)
        data_dict[key] = df    


In [67]:
#data_dict['SF36-104v1.csv']

In [8]:
# (Recode missing by mean) 
column_sets = [
    ['3a', '3b', '3c', '3d', '3e', '3f', '3g', '3h', '3i', '3j'],
    ['4a', '4b', '4c', '4d'],
    ['7', '8'],
    ['1', '11a', '11b', '11c', '11d'],
    ['9a', '9e', '9g', '9i'],
    ['6', '10'],
    ['5a', '5b', '5c'],
    ['9b', '9c', '9d', '9f', '9h']
]

for df_name, df in data_dict.items():
    for columns in column_sets:
        row_mean = df[columns].mean(axis=1)
        for col in columns:
            df[col] = df[col].fillna(row_mean)

#to check
for key, df in data_dict.items():
    missing_values = df[df.isna().any(axis=1)]
    if not missing_values.empty:
        for index, row in missing_values.iterrows():
            missing_columns = row.index[row.isna()]
            print(f"ID {key}: missing in {', '.join(missing_columns)}")

In [9]:
# Compute raw scales
scale_columns = {
    'Physical Functioning': ['3a', '3b', '3c', '3d', '3e', '3f', '3g', '3h', '3i', '3j'],
    'Role-Physical': ['4a', '4b', '4c', '4d'],
    'Bodily-Pain': ['7', '8'],
    'General Health': ['1', '11a', '11b', '11c', '11d'],
    'Vitality': ['9a', '9e', '9g', '9i'],
    'Social Functioning': ['6', '10'],
    'Role-Emotional': ['5a', '5b', '5c'],
    'Mental Health': ['9b', '9c', '9d', '9f', '9h'],
    'Reported Health Transition': ['2'],
    'Mean Current Health': ['1']
}

scale_dict = {}

for key, df in data_dict.items():
    scale_df = pd.DataFrame(columns=scale_columns.keys())
    
    # Calculer les valeurs pour chaque échelle
    for scale, columns in scale_columns.items():
        scale_df[scale] = df[columns].astype(float).sum(axis=1)
    
    scale_dict[key] = scale_df

In [70]:
#scale_dict['SF36-104v1.csv']

In [10]:
# Transform raw scales to 0-100 scales
transformed_scale_dict = {}

for key, df in scale_dict.items():
    df['Physical Functioning'] = (df['Physical Functioning']-10)/20*100
    df['Role-Physical'] = (df['Role-Physical']-4)/4*100
    df['Bodily-Pain'] = (df['Bodily-Pain']-2)/10*100
    df['General Health'] = (df['General Health']-5)/20*100
    df['Vitality'] = (df['Vitality']-4)/20*100
    df['Social Functioning'] = (df['Social Functioning']-2)/8*100
    df['Role-Emotional'] = (df['Role-Emotional']-3)/3*100
    df['Mental Health'] = (df['Mental Health']-5)/25*100
    df['Reported Health Transition'] = (df['Reported Health Transition']-1)/5*100

    transformed_scale_dict[key] = df
    
replacement_dicts = {'Mean Current Health': {5:100, 4.4:84, 3.4:61, 2:25, 1:0}}

# Parcourir chaque DataFrame dans data_dict
for key, df in transformed_scale_dict.items():
    for col, replacement_dict in replacement_dicts.items():
        if col in df.columns:
            df[col] = df[col].replace(replacement_dict)
            transformed_scale_dict[key] = df

In [72]:
#transformed_scale_dict['SF36-104v1.csv']

In [11]:
# (Gather scales) - this is described as a possibility
for key, df in transformed_scale_dict.items():
    df['PHYSICAL'] = df[['Physical Functioning', 
                                         'Role-Physical', 
                                         'Bodily-Pain', 
                                         'General Health']].mean(axis=1)
    df['MENTAL'] = df[['Vitality', 
                                   'Social Functioning', 
                                   'Role-Emotional', 
                                   'Mental Health']].mean(axis=1)
    
    df['GLOBAL'] = df[['Physical Functioning', 'Role-Physical', 'Bodily-Pain',
       'General Health', 'Vitality', 'Social Functioning', 'Role-Emotional',
       'Mental Health']].mean(axis=1)
    
    df.insert(0, 'ID', key)
    
    transformed_scale_dict[key] = df
    
for key in transformed_scale_dict:
    # add the visit
    if 'v3' in key :
        transformed_scale_dict[key]['V'] = '3'
    else :
        transformed_scale_dict[key]['V'] = '1'

    # rephrase ID
    transformed_scale_dict[key]['ID'] = transformed_scale_dict[key]['ID'].str.replace('SF36-', '')
    transformed_scale_dict[key]['ID'] = transformed_scale_dict[key]['ID'].str.replace(r'v.*', '', regex=True)

In [12]:
concatenated_df = pd.concat(transformed_scale_dict.values(), axis=0, ignore_index=True)
concatenated_df

,ID,Physical Functioning,Role-Physical,Bodily-Pain,General Health,Vitality,Social Functioning,Role-Emotional,Mental Health,Reported Health Transition,Mean Current Health,PHYSICAL,MENTAL,GLOBAL,V
0,001,75.0,50.0,51.0,37.0,45.000000,87.5,100.000000,84.0,40.0,61.0,53.25,79.125000,66.187500,1
1,001,90.0,100.0,72.0,77.0,70.000000,100.0,33.333333,88.0,60.0,61.0,84.75,72.833333,78.791667,3
2,002,60.0,100.0,90.0,52.0,46.666667,50.0,100.000000,64.0,20.0,61.0,75.50,65.166667,70.333333,1
3,003,55.0,0.0,41.0,25.0,55.000000,50.0,0.000000,80.0,0.0,25.0,30.25,46.250000,38.250000,1
4,003,80.0,50.0,31.0,72.0,60.000000,50.0,100.000000,64.0,20.0,61.0,58.25,68.500000,63.375000,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
135,105,95.0,100.0,90.0,77.0,80.000000,87.5,100.000000,72.0,20.0,84.0,90.50,84.875000,87.687500,3
136,106,95.0,100.0,74.0,77.0,55.000000,62.5,100.000000,64.0,40.0,61.0,86.50,70.375000,78.437500,1
137,108,85.0,0.0,52.0,35.0,25.000000,25.0,0.000000,40.0,40.0,25.0,43.00,22.500000,32.750000,1
138,108,95.0,50.0,74.0,57.0,60.000000,50.0,66.666667,52.0,20.0,61.0,69.00,57.166667,63.083333,3


In [13]:
concatenated_df.describe()

,Physical Functioning,Role-Physical,Bodily-Pain,General Health,Vitality,Social Functioning,Role-Emotional,Mental Health,Reported Health Transition,Mean Current Health,PHYSICAL,MENTAL,GLOBAL
count,140.000000,140.000000,140.000000,140.000000,140.00000,140.000000,140.000000,140.000000,140.000000,140.000000,140.000000,140.000000,140.000000
mean,78.479167,55.178571,58.800000,49.857143,50.25000,69.553571,65.238095,65.142857,28.857143,57.500000,60.578720,62.546131,61.562426
std,17.643529,37.085948,21.096967,18.143541,14.91104,24.647274,38.115360,18.428775,21.359739,19.955887,17.499712,18.637902,16.873840
min,10.000000,0.000000,0.000000,5.000000,10.00000,0.000000,0.000000,20.000000,0.000000,0.000000,3.750000,22.500000,20.125000
25%,70.000000,25.000000,41.000000,36.500000,40.00000,50.000000,33.333333,52.000000,20.000000,61.000000,47.000000,46.750000,49.338542
50%,83.333333,50.000000,61.000000,52.000000,50.00000,75.000000,66.666667,68.000000,20.000000,61.000000,60.750000,65.604167,63.343750
75%,90.000000,100.000000,74.000000,65.500000,61.25000,90.625000,100.000000,80.000000,40.000000,61.000000,74.611111,78.291667,76.352431
max,100.000000,100.000000,90.000000,90.000000,80.00000,100.000000,100.000000,100.000000,80.000000,100.000000,94.250000,95.000000,90.625000


In [14]:
import os
import pandas as pd
from datetime import datetime

output_path = "/Users/mgg/Library/CloudStorage/GoogleDrive-matthieu.gallou.guyot@gmail.com/.shortcut-targets-by-id/1WxK2x1j2i3mI-QbxfSsy98PtnJYzUPEH/Dakar MGG/Clean Data SF36"

date_today = datetime.today().strftime('%Y-%m-%d')
output_folder = os.path.join(output_path, date_today)

if not os.path.exists(output_folder):
    os.makedirs(output_folder)

for key, df in transformed_scale_dict.items():
    file_path = os.path.join(output_folder, f"{key}.csv")
    df.to_csv(file_path, index=False)
    
concatenated_file_path = os.path.join(output_path, f"concat_{date_today}.csv")
concatenated_df.to_csv(concatenated_file_path, index=False)